In [36]:
import pandas as pd
import  numpy as np
import matplotlib
from tqdm import tqdm
import ray

In [37]:
df = pd.read_csv('distinct_subject_features.csv', dtype={"value": str})

In [38]:
df = df[pd.to_numeric(df['value'], errors='coerce').notnull()]
df['value'] = pd.to_numeric(df["value"], downcast="float")

In [39]:
uniq_subjects = df.subject_id.unique()

In [40]:
df.columns

Index(['subject_id', 'anchor_age', 'gender', 'label', 'value', 'valueuom',
       'flag'],
      dtype='object')

In [ ]:
set_of_labels = set()
for subject_id in tqdm(uniq_subjects):
    uniq_labels = set(df[df['subject_id']==subject_id].label.unique())
    if len(set_of_labels) == 0:
        set_of_labels = uniq_labels
    else:
        set_of_labels = set_of_labels.intersection(uniq_labels)

 43%|██████████████████████████████████                                              | 8999/21167 [04:09<01:58, 103.10it/s]